# Example Usage of Scraipe Library

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import things we need. We will use `NewsScraper` and `OpenAiAnalyzer` from the `scraipe[extras]` subpackage.

We will also load [your OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key) from a file called 'openai_key.txt'. This file should be in the same folder as this notebook.

In [1]:
# Install Scraipe
%pip install scraipe[extras] --quiet --upgrade
!pip show scraipe | grep Version

Note: you may need to restart the kernel to use updated packages.
Version: 0.1.20


In [2]:

# Import modules
import pandas as pd
from scraipe import Workflow
from scraipe.extras import NewsScraper, OpenAiAnalyzer
from pydantic import BaseModel

# Load OpenAI API key
OPENAI_API_KEY = open("openai_key.txt").read().strip()

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [3]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
import time
print(f"Found {len(news_links_df)} front page AP News links on {time.strftime('%Y-%m-%d')}")
display(news_links_df.head())

Found 132 front page AP News links on 2025-04-03


,link
0,https://apnews.com/article/argentina-supreme-c...
1,https://apnews.com/article/trump-tariffs-going...
2,https://apnews.com/article/ken-paxton-ag-feder...
3,https://apnews.com/article/south-korea-martial...
4,https://apnews.com/article/south-korea-enterta...


## Configure Workflow
Now we'll configure the scraipe workflow using `NewsScraper` and `OpenAiAnalyzer`.

`NewsScraper` uses the `trafilatura` to extract article content from a news site without all the HTML clutter.

`OpenAiAnalyzer` uses OpenAI models to extract data from the article content.

In [4]:
#===Configure NewsScraper===
# NewsScraper doesn't require any additional configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''
Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}
'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(OPENAI_API_KEY,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape content from news links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [5]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
scrape_store_df = workflow.get_scrapes()
display(scrape_store_df.head())

Scraping 132/132 new or retry links...


Scraping: 100%|██████████| 132/132 [00:13<00:00, 10.01link/s]

Successfully scraped 132/132 links.


,link,content,scrape_success,scrape_error
0,https://apnews.com/article/united-states-china...,US and Chinese military officials hold talks o...,True,None
1,https://apnews.com/article/space-tourism-north...,Bitcoin investor buys an entire SpaceX flight ...,True,None
2,https://apnews.com/article/oz-hhs-cms-kennedy-...,Senate confirms Mehmet Oz to take lead of Medi...,True,None
3,https://apnews.com/article/measles-outbreak-te...,Measles spreads to central Texas; 5 states hav...,True,None
4,https://apnews.com/article/wisconsin-supreme-c...,Liberal to take over as chief justice on Wisco...,True,None


## Analyze content with OpenAI
Next we will analyze the stored scrapes.

In [6]:
# Analyze the scraped content
workflow.analyze()

# Display the analyses
analysis_store_df = workflow.get_analyses()
display(analysis_store_df.head())

Analyzing 130/130 new or retry links with content...


Analyzing: 100%|██████████| 130/130 [00:41<00:00,  3.17item/s]

Successfully analyzed 130/130 links.


,link,output,analysis_success,analysis_error
0,https://apnews.com/article/alaska-missing-dog-...,{'celebrities': []},True,NaN
1,https://apnews.com/article/trump-new-tariffs-r...,"{'celebrities': ['Donald Trump', 'Josh Boak', ...",True,NaN
2,https://apnews.com/article/diversity-equity-in...,"{'celebrities': ['Donald Trump', 'Martin Luthe...",True,NaN
3,https://apnews.com/article/hoosiers-devries-co...,"{'celebrities': ['Tom Izzo', 'Matt Painter', '...",True,NaN
4,https://apnews.com/article/black-hair-formalde...,"{'celebrities': ['Javon Ford', 'James Rogers',...",True,NaN


## Compile the results
Finally, let's export the completed analysis. 

In [7]:
export_df = workflow.export()
display(export_df)
export_df.to_csv('celebrities.csv', index=False)

,link,scrape_success,analysis_success,celebrities
0,https://apnews.com/article/alaska-missing-dog-...,True,True,[]
1,https://apnews.com/article/trump-new-tariffs-r...,True,True,"[Donald Trump, Josh Boak, Zeke Miller]"
2,https://apnews.com/article/diversity-equity-in...,True,True,"[Donald Trump, Martin Luther King Jr., Mary Fr..."
3,https://apnews.com/article/hoosiers-devries-co...,True,True,"[Tom Izzo, Matt Painter, Brad Underwood, Mike ..."
4,https://apnews.com/article/black-hair-formalde...,True,True,"[Javon Ford, James Rogers, Jasmine McDonald, G..."
...,...,...,...,...
127,https://apnews.com/article/yale-scholar-fired-...,True,True,[]
128,https://apnews.com/article/dementia-alzheimers...,True,True,[]
129,https://apnews.com/article/israel-gaza-hamas-s...,True,True,"[Benjamin Netanyahu, Donald Trump, Yoav Gallant]"
130,https://apnews.com/article/severe-weather-thin...,False,False,NaN


## Analyze the results
Now you can conduct your own analysis on the structured data collected by the scraipe workflow.

In [8]:
# Load the extracted data
celebrities_df = pd.read_csv('celebrities.csv')
from ast import literal_eval
celebrities_df['celebrities'] = celebrities_df['celebrities'].apply(literal_eval)

# Explode the nested list of celebrities
celebrities_df = celebrities_df.explode('celebrities')
celebrities_df['celebrities'] = celebrities_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
celebrities_df = celebrities_df['celebrities'].value_counts().reset_index()
celebrities_df.columns = ['celebrity', 'mentions']
celebrities_df = celebrities_df.sort_values('mentions', ascending=False)
celebrities_df.head(10)

ValueError: malformed node or string: nan